<a href="https://colab.research.google.com/github/MarijanMarkovic/Numerical-modelling-of-differential-equations/blob/main/StationaryNS__vj1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:fenics-packages/fenics
!apt-get update -qq
!apt install -y --no-install-recommends fenics
!rm -rf

In [ ]:
from fenics import*

#1. Mesh Generation
nx, ny=20, 20
mesh=UnitSquareMesh(nx, ny, 'crossed')

#2. Finite Element Space
degree=1
V=VectorElement('CG', mesh.ufl_cell(), degree+1)
Q=FiniteElement('CG', mesh.ufl_cell(), degree)

X=FunctionSpace(mesh, MixedElement([V, Q]))

u_boundary=Expression(('near(x[1], 1) ? 1.0 : 0.0', '0'), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [
    DirichletBC(X.sub(0), u_boundary, boundary),
    DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')
]

#3. Problem Definition
def solve_stationary_NS(X, advection, Re, f, bc):
  u, p=TrialFunctions(X)
  v, q=TestFunctions(X)
  
  a=(inner(grad(u), grad(v))/Re - div(v) * p  - q * div(u)) * dx
  L=dot(f, v) * dx
  
  if advection:
    a +=dot(grad(u) * advection, v) * dx
    
  x=Function(X)
  solve(a==L, x, bc)
  
  return x.split()

#4. Solution
Re=Constant(300)
f=Constant((0,0))

uh, ph=solve_stationary_NS(X, None, Re, f, bc)

niter=100
tolerance=1e-6
for i in range(niter):
  uh_old, ph_old=uh, ph
  uh, ph=solve_stationary_NS(X, uh_old, Re, f, bc)
  
  error=(errornorm(uh, uh_old, 'H1')/norm(uh_old, 'H1')+
         errornorm(ph, ph_old, 'L2')/norm(ph_old, 'L2'))
  
  print('step {}: {:.3e}'.format(i, error))
  if error<tolerance:
    break
    
uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('stationaryNS_velocity.pvd') << uh
File('stationaryNS_pressure.pvd') << ph

Calling FFC just-in-time (JIT) compiler, this may take some time.
step 0: 2.797e+00
step 1: 4.765e-01
step 2: 1.248e-01
step 3: 2.813e-02
step 4: 7.301e-03
step 5: 1.794e-03
step 6: 9.377e-04
step 7: 5.126e-04
step 8: 1.769e-04
step 9: 7.954e-05
step 10: 4.064e-05
step 11: 1.783e-05
step 12: 8.430e-06
step 13: 4.097e-06
step 14: 1.900e-06
step 15: 8.954e-07


In [ ]:
#!/usr/bin/env python

from fenics import *

# 1. mesh generation
nx, ny = 10, 10
mesh = UnitSquareMesh(nx, ny, 'crossed')

# 2. finite element space
degree = 1
V = VectorElement('CG', mesh.ufl_cell(), degree+1)
Q = FiniteElement('CG', mesh.ufl_cell(), degree)

X = FunctionSpace(mesh, MixedElement([V, Q]))

u_boundary = Expression((
        'near(x[1], 1) ? 1.0 : 0.0',
        '0'
    ), degree=0)

def boundary(x, on_boundary):
  return on_boundary

def origin(x, on_boundary):
  return near(x[0], 0) and near(x[1], 0)

bc = [
    DirichletBC(X.sub(0), u_boundary, boundary),
    DirichletBC(X.sub(1), Constant(0), origin, 'pointwise')
]

# 3. problem definition
def solve_linear(X, advection, Re, f, bc):
  u, p = TrialFunctions(X)
  v, q = TestFunctions(X)
  
  a = (inner(grad(u), grad(v)) / Re - p * div(v) - div(u) * q) * dx
  L = dot(f, v) * dx
  if advection:
    a += dot(grad(u) * advection, v) * dx

    A = lambda u, p: -div(grad(u)) / Re + (grad(u) * advection) + div(advection) * u / 2 + grad(p)
    A_SS = lambda u, p: (grad(u) * advection) + div(advection) * u / 2 + grad(p)
  
    h = CellDiameter(X.mesh())
    anorm = sqrt(dot(advection, advection))
    tau_K = 0.5 * h / conditional(anorm * h * Re > 2, anorm, 2 / h / Re)
  
    a += tau_K * (dot(A(u, p), A_SS(v, q)) + div(u) * div(v)) * dx
    L += tau_K * (dot(f, A_SS(v, q))) * dx
  
  x = Function(X)
  solve(a == L, x, bc)
  
  return x.split()

# 4. solution
Re = Constant(3000)
f = Constant((0, 0))

uh, ph = solve_linear(X, Constant((0, 0)), Re, f, bc)

niter = 40
tolerance = 1e-6
for i in range(niter):
  uh_old, ph_old = uh, ph
  uh, ph = solve_linear(X, uh_old, Re, f, bc)
  
  error = (errornorm(uh, uh_old, 'H1') / norm(uh_old, 'H1') +
           errornorm(ph, ph_old, 'L2') / norm(ph_old, 'L2'))
  print('step {}: {:.3e}'.format(i, error))
  if error < tolerance:
    break

uh.rename('velocity', 'velocity')
ph.rename('pressure', 'pressure')

File('velocity.pvd') << uh
File('pressure.pvd') << ph


step 9: 5.694e-02
step 10: 3.460e-02
step 11: 2.504e-02
step 12: 1.691e-02
step 13: 1.156e-02
step 14: 7.971e-03
step 15: 5.383e-03
step 16: 3.733e-03
step 17: 2.514e-03
step 18: 1.747e-03
step 19: 1.177e-03
step 20: 8.182e-04
step 21: 5.520e-04
step 22: 3.832e-04
step 23: 2.591e-04
step 24: 1.796e-04
step 25: 1.216e-04
step 26: 8.417e-05
step 27: 5.712e-05
step 28: 3.946e-05
step 29: 2.682e-05
step 30: 1.851e-05
step 31: 1.260e-05
step 32: 8.681e-06
step 33: 5.915e-06
step 34: 4.073e-06
step 35: 2.778e-06
step 36: 1.911e-06
step 37: 1.304e-06
step 38: 8.968e-07


In [ ]:
plot(uh)